In [10]:
using Flux, Metalhead, Images
using Flux: @epochs
using StatsBase: sample, shuffle
using Flux.Data: DataLoader
using Flux: onehotbatch

# Boats Dataset

## Read files and label

In [3]:
const PATH = joinpath(@__DIR__, "/media/hdd/Datasets/imagewoof2-160/train/")
const FILES = joinpath.(PATH, readdir(PATH))
if isempty(readdir(PATH))
  error("Empty train folder - perhaps you need to download and extract the kaggle dataset.")
end

In [4]:
all_files = reduce(vcat, readdir(joinpath(PATH,i), join=true) for i in readdir(PATH));

In [5]:
boats_y_func(fpath) = split(fpath, "/")[end-1]

boats_y_func (generic function with 1 method)

In [6]:
boats_y = map(boats_y_func, all_files); # get all y labels from filepath
unique_classes = unique(boats_y); # unique classes
n_class = length(unique_classes); # num of classes
dict_labels = Dict((unique_classes[y]) => y for y in 1:n_class)

Dict{SubString{String}, Int64} with 10 entries:
  "n02115641" => 10
  "n02099601" => 7
  "n02088364" => 3
  "n02105641" => 8
  "n02087394" => 2
  "n02086240" => 1
  "n02111889" => 9
  "n02089973" => 4
  "n02093754" => 5
  "n02096294" => 6

In [7]:
resnet = ResNet().layers

Chain(
  Chain([
    Conv((7, 7), 3 => 64, pad=3, stride=2, bias=false),  # 9_408 parameters
    BatchNorm(64, relu),                # 128 parameters, plus 128
    MaxPool((3, 3), pad=1, stride=2),
    Parallel(
      Metalhead.addrelu,
      Chain(
        Conv((1, 1), 64 => 64, bias=false),  # 4_096 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
        Conv((3, 3), 64 => 64, pad=1, bias=false),  # 36_864 parameters
        BatchNorm(64, relu),            # 128 parameters, plus 128
        Conv((1, 1), 64 => 256, bias=false),  # 16_384 parameters
        BatchNorm(256),                 # 512 parameters, plus 512
      ),
      Chain([
        Conv((1, 1), 64 => 256, bias=false),  # 16_384 parameters
        BatchNorm(256),                 # 512 parameters, plus 512
      ]),
    ),
    Parallel(
      Metalhead.addrelu,
      Chain(
        Conv((1, 1), 256 => 64, bias=false),  # 16_384 parameters
        BatchNorm(64, relu),            # 128 parameters

In [8]:
model = Chain(
  resnet[1:end-2],
  Dense(2048, 1000),  
  Dense(1000, 256),
  Dense(256, 2),        # we get 2048 features out, and we have 2 classes
)

Chain(
  Chain(),
  Dense(2048 => 1000),                  # 2_049_000 parameters
  Dense(1000 => 256),                   # 256_256 parameters
  Dense(256 => 2),                      # 514 parameters
)                   # Total: 6 arrays, 2_305_770 parameters, 8.796 MiB.

In [9]:
model = model |> gpu


Chain(
  Chain(),
  Dense(2048 => 1000),                  # 2_049_000 parameters
  Dense(1000 => 256),                   # 256_256 parameters
  Dense(256 => 2),                      # 514 parameters
)                   # Total: 6 arrays, 2_305_770 parameters, 856 bytes.

In [11]:
nsize = (224, 224)

(224, 224)

In [ ]:
img_paths = shuffle(all_files)

dict_value(x) = dict_labels[boats_y_func(x)]

labels = map(dict_value, img_paths)

# Load all of the images
imgs = Images.load.(img_paths)

# Re-size the images based on imagesize from above (most models use 224 x 224)
imgs = map(img -> Images.imresize(img, nsize...), imgs)

# Change the dimensions of each image, switch to gray scale. Channel view switches to...
# a 3 channel 3rd dimension and then (3,2,1) makes those into seperate arrays.
# So we end up with [:, :, 1] being the Red values, [:, :, 2] being the Green values, etc
# imgs = map(img -> permutedims(channelview(img), (3,2,1)), imgs)
# Result is two 3D arrays representing each image

# Concatenate the two images into a single 4D array and add another extra dim at the end
# which shows how many images there are per set, in this case, it's 2
imgs = cat(imgs..., dims = 4)


In [42]:
function load_batch(all_files, dict_labels,batchsize=10, nsize = (224, 224))
    if ((batchsize % 2) != 0)
      print("Batch size must be an even number")
    end

    img_paths = shuffle(all_files)

    dict_value(x) = dict_labels[boats_y_func(x)]

    labels = map(dict_value, img_paths)

    # Load all of the images
    imgs = Images.load.(img_paths)
    
    # Re-size the images based on imagesize from above (most models use 224 x 224)
    imgs = map(img -> Images.imresize(img, nsize...), imgs)
    
    # Change the dimensions of each image, switch to gray scale. Channel view switches to...
    # a 3 channel 3rd dimension and then (3,2,1) makes those into seperate arrays.
    # So we end up with [:, :, 1] being the Red values, [:, :, 2] being the Green values, etc
    # imgs = map(img -> permutedims(channelview(img), (3,2,1)), imgs)
    # Result is two 3D arrays representing each image
    
    # Concatenate the two images into a single 4D array and add another extra dim at the end
    # which shows how many images there are per set, in this case, it's 2
    imgs = cat(imgs..., dims = 4)
    # This is requires since the model's input is a 4D array
    
    # Convert the images to float form and return them along with the labels
    # The default is float64 but float32 is commonly used which is why we use it
    Float64.(imgs), labels
end


In [2]:
dataset = [gpu.(load_batch(all_files[1:100], dict_labels, 6)) for i in 1:6]

UndefVarError: UndefVarError: all_files not defined

In [ ]:
opt = ADAM()
loss(x,y) = Flux.Losses.logitcrossentropy(model(x), y)

In [ ]:
ps = Flux.params(model[2:end])  # ignore the already trained layers of the ResNet

In [ ]:
@epochs 2 Flux.train!(loss, ps, dataset, opt)

In [15]:
imgs, labels = gpu.(load_batch(all_files[1:5], dict_labels, 2))
display(model(imgs))

MethodError: MethodError: no method matching (::Dense{typeof(identity), CUDA.CuArray{Float32, 2, CUDA.Mem.DeviceBuffer}, CUDA.CuArray{Float32, 1, CUDA.Mem.DeviceBuffer}})(::Nothing)
Closest candidates are:
  (::Dense)(!Matched::AbstractVecOrMat) at ~/.julia/packages/Flux/KkC79/src/layers/basic.jl:170
  (::Dense)(!Matched::AbstractArray) at ~/.julia/packages/Flux/KkC79/src/layers/basic.jl:175

In [ ]:
labels